# (2) Train AFA agent
The AFA problem contains the training of an AFA agent that can recommends which features to acquire. 
In this notebook we train and save such an AFA agent.

In [1]:
%load_ext autoreload
%autoreload 2

## Define paths

Paths for data

In [2]:
from afa.configurations.utils_ts import specify_default_paths_ts
# which dataset to work on 
dataset_name   = "miiv_test"

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# automatically specify some path locations (change paths manually if needed) 
data_root = '/home2/joshua.wendland/Documents/afa_mi_policies/data/'
paths = specify_default_paths_ts(dataset_name = dataset_name , miss_scenario = miss_scenario, data_root=data_root)

Paths for model

In [3]:
# name for agent (and predictor) 
agent_name            = 'DQN'
predictor_model_name  = 'logistic_regression'
predictor_model_name  = 'TCN'

# new (where to save the model) 
agent_dir           = paths['data_dir']  + 'afa_agents' + '/' + agent_name + '/'
predictor_model_dir = paths['data_dir']  + 'predictor_models' + '/' + predictor_model_name + '/'

## Load dataset with missingness 
At first, we want to load the dataset 

Includes loading: 
- superfeature mapping
- problem
- afa_problem 
- missingness_model

In [4]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts

2023-08-23 13:24:10.617082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 13:24:11.090100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-23 13:24:11.090114: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-23 13:24:17.285431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
data_loader = DataLoader_ts(     data_file                  = paths['data_file'],
                                 temporal_data_file         = paths['temporal_data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'], 
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

Padding sequences: 100%|██████████| 100/100 [00:00<00:00, 1879.81it/s]


In [6]:
# define predictor (has to be pretrained) 
predictor_model_params = {
    'name' : predictor_model_name, 
    'directory': predictor_model_dir,
    'base_model_params' : {   
        'model_type': 'impute_then_regress',
        'imputer_params' : {
              'model_type': 'simple_imputer',
              'mode' : 'imputation', 
            },
        'predictor_params' : {
              'model_type': 'LogisticRegression',
              'mode' : 'classification'
          }
    }
}

predictor_model_params = {
    'name' : predictor_model_name, 
    'directory': predictor_model_dir,
    'base_model_params' : { 
        'model_type': 'TCN',
        'mode': 'classification',   
        'seed': 54,                     # seed for reproduceability
        'lr': 0.01,                     # initial learning rate for lr-scheduler
        'time excluded': True,          # for logging purposes, if time is included as a feature
        'num_inputs': 48,                # The number of features that are used as input
        'num_layers': 2,                # The number of temporal conv. layers
        'channels_per_layer': 8,        # The number of channels per temp. conv. layer
        'kernel_size': 3,               # Num time steps for kernel window
        'max_epochs': 20,              # Max. epochs for training
        'batch_size': 2000,             # Number of samples per batch
        'dropout': 0.3,                 # Dropout for all temp. conv. layers
        'patience': 60,                 # Num epochs as patience for early stopping, default 50.
        'weight_decay': 0,              # Weight decay for L2 regularization of Adam optmizer
        'wandb_logger': False,           # If weights and biases logger should be used
        #'wandb_project_name': 'Synthetic_2 classifier (AFA)'    # project name if wandb_logger is used
        
    }
}

In [7]:
# define agent
agent_model_params =  {                       
                    'num_iterations'              : 100,
                    'initial_collect_steps'       : 50,
                    'collect_steps_per_iteration' : 1, 
                    'replay_buffer_max_length'    : 100000 ,
                    'batch_size'                  : 64,
                    'learning_rate'               : 1e-3,
                    'train_steps_per_iteration'   : 1, 
                    'log_interval'                : 200,
                    'num_eval_episodes'           : 50,
                    'eval_interval'               : 500, 
                    'verbose'                     : False
                      }               

In [8]:
# define afa_agent
afa_agent_params = {
    'name' : agent_name,
    'agent_type' : 'dqn_agent',
    'directory' : agent_dir, 
    'predictor_params' : predictor_model_params, 
    'agent_params' : agent_model_params
}

## Initialize Agent 

In [9]:
from afa.afa_models.afa_agents.utils_ts import define_afa_agent_ts

In [10]:
afa_agent = define_afa_agent_ts(    name             = afa_agent_params['name'],  
                                    agent_type       = afa_agent_params['agent_type'],   
                                    afa_agent_params = afa_agent_params ,
                                    agent_directory  = afa_agent_params['directory'])

Instantiating the TCN_Lightning model...
Conv1d layer initialized with Kaiming Uniform.
MLP part input size:  8
Value recognized as pad value: -1
Debug: Using Tensorboard logger.
{'batch_size': 2000,
 'channels_per_layer': 8,
 'directory': '/home2/joshua.wendland/Documents/afa_mi_policies/data/miiv_test/MCAR_1/predictor_models/TCN/',
 'dropout': 0.3,
 'kernel_size': 3,
 'lr': 0.01,
 'max_epochs': 20,
 'mode': 'classification',
 'model_type': 'TCN',
 'num_inputs': 48,
 'num_layers': 2,
 'patience': 60,
 'seed': 54,
 'time excluded': True,
 'wandb_logger': False,
 'weight_decay': 0}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Loaded model from checkpoint: /home2/joshua.wendland/Documents/afa_mi_policies/data/miiv_test/MCAR_1/predictor_models/TCN/model_weights/best_model-val_loss=1.55-epoch=2.ckpt


## Train agent

In [11]:
print('gdrf')

gdrf


In [15]:
afa_agent.fit(dataset, 
              fold = 0, train_split = "train", valid_split = "val" , 
              fit_again = True) 

Start training DQN agent


2023-08-23 13:29:09.324170: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: IndexError: index -2147483648 is out of bounds for axis 1 with size 2
Traceback (most recent call last):

  File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/tf_agents/environments/tf_py_environment.py", line 302, in _isolated_step_py
    return self._execute(_step_py, *flattened_actions)

  File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/tf_agents/environments/tf_py_environment.py", line 211, in _execute
    return fn(*args, **kwargs)

  File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/p

UnknownError: Graph execution error:

Detected at node 'driver_loop/step/step_py_func' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home2/joshua.wendland/Documents/afa_ts/afa_env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
  

## Plot training 

In [12]:
logdir = afa_agent_params['agent_params']['directory']
%load_ext tensorboard
%tensorboard --logdir=$logdir --port=3019